In [1]:
import subprocess
import os
import time
import threading
import IPython.display as display
import ipywidgets as widgets

import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt


In [2]:
# -----------------------
# SETTINGS
# -----------------------
script = "parallel_string_sizing.py"
log_file = "log.txt"

# -----------------------
# LAUNCH SCRIPT
# -----------------------
with open(log_file, "w") as logfile:
    process = subprocess.Popen(
        ["python", script],
        stdout=logfile,
        stderr=subprocess.STDOUT
    )

pid = process.pid
print(f"🚀 Script started with PID: {pid}. Output is logging to: {log_file}")

# -----------------------
# STOP BUTTON
# -----------------------
stop_btn = widgets.Button(description="🛑 Stop Script", button_style='danger')

def stop_script(b):
    process.terminate()
    print(f"🔴 Terminated script with PID {pid}")

stop_btn.on_click(stop_script)
display.display(stop_btn)

# -----------------------
# LIVE LOG DISPLAY (last 10 lines)
# -----------------------
log_output = widgets.Output()
display.display(log_output)

def update_log_display():
    while process.poll() is None:
        with log_output:
            log_output.clear_output(wait=True)
            try:
                with open(log_file, "r") as f:
                    lines = f.readlines()[-10:]
                    print("".join(lines))
            except:
                print("Waiting for log...")
        time.sleep(2)

# Start background thread to monitor log
threading.Thread(target=update_log_display, daemon=True).start()


🚀 Script started with PID: 49603. Output is logging to: log.txt


Button(button_style='danger', description='🛑 Stop Script', style=ButtonStyle())

Output()

In [ ]:
# Load your simulation results
df = pd.read_csv("string_sizing_results.csv")

# Preview
df.sample(10)


In [ ]:
df["ExceedsLimit"] = df["VocMax"] > 1500


In [ ]:
# Create a pivot table of max Voc across all years
pivot = df.pivot_table(index='Module',
                       columns='ModulesPerString',
                       values='VocMax',
                       aggfunc='max')

# Optional: sort modules alphabetically
pivot = pivot.sort_index()


In [ ]:
# Add a helper column to highlight rows with any Voc > 1500V
pivot["Any_Over_1500V"] = (pivot > 1500).any(axis=1)


In [ ]:
pivot.to_csv("pivot_summary_by_module.csv")
print("✅ Saved: pivot_summary_by_module.csv")
pivot.head(10)


In [ ]:
# Load into DataFrame
import seaborn as sns
import matplotlib.pyplot as plt


# Set seaborn style
sns.set(style="darkgrid")

# FacetGrid with lineplot
g = sns.FacetGrid(df, col="Module", col_wrap=1, height=5, aspect=1.5)
g.map_dataframe(sns.lineplot, x="Year", y="VocMax", hue="ModulesPerString", palette="tab10", marker="o")

# Add legend and labels
g.add_legend(title="ModulesPerString")
g.set_titles(row_template="{row_name}")
g.set_axis_labels("Year", "VocMax")
plt.tight_layout()
plt.show()


In [ ]:
f, ax = plt.subplots(figsize=(7, 5))
sns.despine(f)

sns.histplot(
    df,
    x="VocMax", hue="ModulesPerString",
    multiple="stack",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    log_scale=True,
)
ax.xaxis.set_major_formatter(mpl.ticker.ScalarFormatter())
# ax.set_xticks([500, 1000, 2000, 5000, 10000])